In [ ]:
from statsbombpy import sb
import mplsoccer

import pandas as pd
import numpy as np
import math
import ast
import time

import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm

In [ ]:
sb.competitions().sort_values('season_name').tail(20)

### Getting the data availible from 2020 to 2022

In [ ]:
ISL = sb.matches(competition_id=1238,season_id=108)

In [ ]:
ISL = ISL[ISL['match_status']=='available']

In [ ]:
ISL

In [ ]:
ISL.columns

In [ ]:
matches_list = ISL['match_id']

In [ ]:
def in_box(player):
    
    x,y = player['location']
    #print('in_box works')
    return True if ((102<=x<=120) and (18<=y<=62)) else False
    

In [ ]:
def ahead(player,shot_location):
    
    x1,y1 = shot_location
    x,y = player['location']
    #print('ahead works')
    return True if (x1<=x<=120) else False

In [ ]:
def near_shooter(player,shot_location):
    x1,y1 = shot_location
    x,y = player['location']
    dist = np.sqrt((x1-x)**2 + (y1-y)**2)
    #print('near-shooter works')
    return True if dist<=2 else False

In [ ]:
def is_teammate(player):
    return player['teammate']

In [ ]:
def is_gk(player):
    pos = player['position']['name']
    #print(pos)
    return (pos == 'Goalkeeper')


In [ ]:
def dist_from_goal(player):
    x,y = player['location']
    #print('dist_from_goal works')
    return np.sqrt((120-x)**2 + (40-y)**2)

In [ ]:
def player_locations(players,location):
    teammates_in_box = 0
    teammates_ahead = 0
    opp_in_box = 0
    gk_dist_from_goal = None
    gk_x = None
    gk_y = None
    opponents_near = 0
    #print('initialized')
    for player in players:
        if is_teammate(player):
            if ahead(player,location):
                teammates_ahead+=1
            if in_box(player):
                teammates_in_box+=1
            #print('teammate works')
        else:
            if is_gk(player):
                #print('is_gk')
                gk_x = player['location'][0]
                gk_y = player['location'][1]
                gk_dist_from_goal = dist_from_goal(player)
            else:
                #print('opponent')
                if in_box(player):
                    opp_in_box +=1
                if near_shooter(player,location):
                    opponents_near+=1
                    
    return teammates_in_box,teammates_ahead,opp_in_box,gk_dist_from_goal,gk_x,gk_y,opponents_near
                
            
        

In [ ]:
shots_data = []
pbar = tqdm(total=len(matches_list))

for idx in matches_list:
    #time.sleep(10)
    events = sb.events(match_id=idx, split=True, flatten_attrs=False)["shots"]
    events = events[events['play_pattern']=='Regular Play'][['id', 'index','play_pattern','location','shot','possession_team']].reset_index()
    events['type'] = events.apply(lambda x:x['shot']['type']['name'],axis=1)
    events['outcome'] = events.apply(lambda x:x['shot']['outcome']['name'],axis=1)
    events['body_part'] = events.apply(lambda x:x['shot']['body_part']['name'],axis=1)
    events['players'] = events.apply(lambda x:x['shot']['freeze_frame'],axis=1)
    home_team = ISL.loc[ISL['match_id']==idx,'home_team'].values[0]
    events['home_team'] = events.apply(lambda x: 1 if x['possession_team']==home_team else 0,axis=1)
    events['follows_dribble'] = events.apply(lambda x: 1 if 'follows_dribble' in x['shot'].keys() else 0,axis=1)
    events2 = events.apply(lambda x:player_locations(x['players'],x['location']),axis=1)
    events2 = pd.DataFrame(list(events2),columns=['teammates_in_box','teammates_ahead','opp_in_box','gk_dist_from_goal','gk_x','gk_y','opponents_near'])
    events = pd.concat([events,events2],axis=1)
    shots_data.append(events)
    pbar.update(1)
    
pbar.close

In [ ]:
shots_data

In [ ]:
ISL_shots = pd.concat(shots_data)

In [ ]:
ISL_shots.to_csv('D:/Football_data_scientist/data/ISL_shots_added_features.csv')

In [ ]:
LaLiga_20_21 = sb.matches(competition_id=11,season_id=90)
LaLiga_20_21 = LaLiga_20_21[LaLiga_20_21['match_status']=='available']
LaLiga_20_21_matches_list = LaLiga_20_21['match_id']

In [ ]:
shots_data_LaLiga_20_21 = []
pbar = tqdm(total=len(LaLiga_20_21_matches_list))

for idx in LaLiga_20_21_matches_list:
    
    events = sb.events(match_id=idx, split=True, flatten_attrs=False)["shots"]
    events = events[events['play_pattern']=='Regular Play'][['id', 'index','play_pattern','location','shot','possession_team']].reset_index()
    events['type'] = events.apply(lambda x:x['shot']['type']['name'],axis=1)
    events['outcome'] = events.apply(lambda x:x['shot']['outcome']['name'],axis=1)
    events['body_part'] = events.apply(lambda x:x['shot']['body_part']['name'],axis=1)
    events['players'] = events.apply(lambda x:x['shot']['freeze_frame'],axis=1)
    #events['home_team'] = events.apply(lambda x: 1 if x['possession_team']==home_team else 0,axis=1)
    events['follows_dribble'] = events.apply(lambda x: 1 if 'follows_dribble' in x['shot'].keys() else 0,axis=1)
    events2 = events.apply(lambda x:player_locations(x['players'],x['location']),axis=1)
    events2 = pd.DataFrame(list(events2),columns=['teammates_in_box','teammates_ahead','opp_in_box','gk_dist_from_goal','gk_x','gk_y','opponents_near'])
    events = pd.concat([events,events2],axis=1)
    shots_data_LaLiga_20_21.append(events)
    pbar.update(1)
    
pbar.close

In [ ]:
LaLiga_20_21_shots = pd.concat(shots_data_LaLiga_20_21)

In [ ]:
LaLiga_20_21_shots.to_csv('LaLiga_20_21_shots.csv')

In [ ]:
LaLiga_19_20 = sb.matches(competition_id=11,season_id=42)
LaLiga_19_20 = LaLiga_19_20[LaLiga_19_20['match_status']=='available']
LaLiga_19_20_matches_list = LaLiga_19_20['match_id']


In [ ]:
shots_data_LaLiga_19_20 = []
pbar = tqdm(total=len(LaLiga_19_20_matches_list))

for idx in LaLiga_19_20_matches_list:
    events = sb.events(match_id=idx, split=True, flatten_attrs=False)["shots"]
    events = events[events['play_pattern']=='Regular Play'][['id', 'index','play_pattern','location','shot','possession_team']].reset_index()
    events['type'] = events.apply(lambda x:x['shot']['type']['name'],axis=1)
    events['outcome'] = events.apply(lambda x:x['shot']['outcome']['name'],axis=1)
    events['body_part'] = events.apply(lambda x:x['shot']['body_part']['name'],axis=1)
    events['players'] = events.apply(lambda x:x['shot']['freeze_frame'],axis=1)
    events['home_team'] = events.apply(lambda x: 1 if x['possession_team']==home_team else 0,axis=1)
    events['follows_dribble'] = events.apply(lambda x: 1 if 'follows_dribble' in x['shot'].keys() else 0,axis=1)
    events2 = events.apply(lambda x:player_locations(x['players'],x['location']),axis=1)
    events2 = pd.DataFrame(list(events2),columns=['teammates_in_box','teammates_ahead','opp_in_box','gk_dist_from_goal','gk_x','gk_y','opponents_near'])
    events = pd.concat([events,events2],axis=1)
    shots_data_LaLiga_19_20.append(events)
    pbar.update(1)
    
pbar.close

In [ ]:
LaLiga_19_20_shots = pd.concat(shots_data_LaLiga_19_20)



In [ ]:
LaLiga_19_20_shots.to_csv('LaLiga_19_20_shots_extended.csv')


In [ ]:
Euro_2020 = sb.matches(competition_id=55,season_id=43)
Euro_2020 = Euro_2020[Euro_2020['match_status']=='available']
Euro_2020_matches_list = Euro_2020['match_id']

In [ ]:
shots_data_Euro_2020 = []
pbar = tqdm(total=len(Euro_2020_matches_list))

for idx in Euro_2020_matches_list:
    events = sb.events(match_id=idx, split=True, flatten_attrs=False)["shots"]
    events = events[events['play_pattern']=='Regular Play'][['id', 'index','play_pattern','location','shot','possession_team']].reset_index()
    events['type'] = events.apply(lambda x:x['shot']['type']['name'],axis=1)
    events['outcome'] = events.apply(lambda x:x['shot']['outcome']['name'],axis=1)
    events['body_part'] = events.apply(lambda x:x['shot']['body_part']['name'],axis=1)
    events['players'] = events.apply(lambda x:x['shot']['freeze_frame'],axis=1)
    #events['home_team'] = events.apply(lambda x: 1 if x['possession_team']==home_team else 0,axis=1)
    events['follows_dribble'] = events.apply(lambda x: 1 if 'follows_dribble' in x['shot'].keys() else 0,axis=1)
    events2 = events.apply(lambda x:player_locations(x['players'],x['location']),axis=1)
    events2 = pd.DataFrame(list(events2),columns=['teammates_in_box','teammates_ahead','opp_in_box','gk_dist_from_goal','gk_x','gk_y','opponents_near'])
    events = pd.concat([events,events2],axis=1)
    shots_data_Euro_2020.append(events)
    pbar.update(1)
    
pbar.close

Euro_2020_shots = pd.concat(shots_data_Euro_2020)

In [ ]:
Euro_2020_shots.to_csv('Euro_2020_shots_extended.csv')

In [ ]:
Fifa_2022 = sb.matches(competition_id=43,season_id=106)
Fifa_2022 = Fifa_2022[Fifa_2022['match_status']=='available']
Fifa_2022_matches_list = Fifa_2022['match_id']




In [ ]:
Fifa_2022_matches_list[13]

In [ ]:
events = sb.events(match_id=Fifa_2022_matches_list[13], split=True, flatten_attrs=False)["shots"]
events = events[events['play_pattern']=='Regular Play'][['id', 'index','play_pattern','location','shot']].reset_index()
events

In [ ]:
shots_data_Fifa_2022 = []
pbar = tqdm(total=len(Fifa_2022_matches_list))

for idx in Fifa_2022_matches_list:
    events = sb.events(match_id=idx, split=True, flatten_attrs=False)["shots"]
    events = events[events['play_pattern']=='Regular Play'][['id', 'index','play_pattern','location','shot']].reset_index()
    if len(events)>0:
        events['type'] = events.apply(lambda x:x['shot']['type']['name'],axis=1)
        events['outcome'] = events.apply(lambda x:x['shot']['outcome']['name'],axis=1)
        events['body_part'] = events.apply(lambda x:x['shot']['body_part']['name'],axis=1)
        shots_data_Fifa_2022.append(events)
    pbar.update(1)
    
pbar.close

Fifa_2022_shots = pd.concat(shots_data_Fifa_2022)



Fifa_2022_shots.to_csv('Fifa_2022_shots.csv')
